In [5]:
!conda install -c pytorch pytorch torchvision torchaudio cpuonly scikit-learn

^C


In [ ]:
!pip install polars

In [ ]:
# References
# https://scikit-learn.org/stable/modules/naive_bayes.html
# https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

In [1]:
import torch
import sklearn

In [ ]:
from sklearn.datasets import fetch_20newsgroups
import polars as pl

# Load data
categories = ['alt.atheism', 'soc.religion.christian','comp.graphics', 'sci.med']
data = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

#  Create dataframe
df = pl.DataFrame(data.data)
df.insert_column(len(df.columns),pl.Series(data.target))

# Set labels
labels = list(data.feature_names)
labels.append("class")
df.columns = labels

In [ ]:
df.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)